In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dataframe=pd.read_csv("DJIA.csv")

In [3]:
dataframe['DJIA']=dataframe['DJIA'].convert_objects(convert_numeric=True)

C:\Users\qinke\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [4]:
dataframe.head()

,DATE,DJIA
0,2017-01-02,NaN
1,2017-01-03,19881.76
2,2017-01-04,19942.16
3,2017-01-05,19899.29
4,2017-01-06,19963.80


In [5]:
dataframe=dataframe.dropna(axis=1)

In [6]:
dataframe.reset_index(drop=True, inplace=True)

In [7]:
print(dataframe['DJIA'])

KeyError: 'DJIA'

dataframe2=dataframe2.drop('label',axis=1)
dataframe2=dataframe2.drop('Unnamed: 0',axis=1)
dataframe2=dataframe2.dropna()

dataframe2=dataframe2.groupby('Date',as_index=False, sort=True).agg({'text':lambda x:' '.join(x),'approval_date':lambda x:x.max()})

print(dataframe2)

In [ ]:
#!/usr/bin/env python
# Implementation of algorithm from https://stackoverflow.com/a/22640362/6029703
import numpy as np
import pylab

def thresholding_algo(y, lag, threshold, influence):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
# Data
y =dataframe['DJIA']

# Settings: lag = 30, threshold = 5, influence = 0
lag = 3
threshold = 1.5
influence = 0.5

# Run algo with settings from above
result = thresholding_algo(y, lag=lag, threshold=threshold, influence=influence)

# Plot result
pylab.subplot(211)
pylab.plot(np.arange(1, len(y)+1), y)

pylab.plot(np.arange(1, len(y)+1),
           result["avgFilter"], color="cyan", lw=2)

pylab.plot(np.arange(1, len(y)+1),
           result["avgFilter"] + threshold * result["stdFilter"], color="green", lw=2)

pylab.plot(np.arange(1, len(y)+1),
           result["avgFilter"] - threshold * result["stdFilter"], color="green", lw=2)

pylab.subplot(212)
pylab.step(np.arange(1, len(y)+1), result["signals"], color="red", lw=2)
pylab.ylim(-1.5, 1.5)

In [ ]:
print(result["signals"])

In [ ]:
list(result["signals"].shape)

In [ ]:
final_result=pd.DataFrame({'Date':dataframe['DATE'],'label':list(result['signals'])})

In [ ]:
final_result

In [ ]:
from collections import Counter
final_result['label'][0]=1
Counter(final_result['label'])

In [ ]:
#Counter(final_result['label'])

final_result.to_csv("signal.csv", index = False)

In [ ]:
print(final_result)

In [ ]:
dataframe2=pd.read_csv("signal.csv")

In [ ]:
del dataframe2['label']

In [ ]:
print(dataframe2)

In [ ]:
new_df=pd.merge(final_result, dataframe2, on='Date')

In [ ]:
print(new_df)

In [ ]:
new_df.to_csv("stock predict signal.csv", index = False)